# CHAKRA SAMHITA

### DISEASE DETECTION AND AYURVEDIC MEDICINE RECOMMENDATION SYSTEM




DIYA VERMA 201302020

IMPORT LIBRARIES

In [56]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from fuzzywuzzy import fuzz
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import os
from tkinter import filedialog
import folium
from PIL import Image, ImageTk, ImageDraw, ImageFont
import csv
import folium

LOADING DATASET:

                DISEASE DETECTION
                MEDICINE RECOMMENDATION

In [57]:
# Load the datasets
data = pd.read_excel('C:/Users/Dell/OneDrive/Desktop/charak_samita_app_sam/Datatset/data1.xlsx')
ayurvedic_data = pd.read_excel('C:/Users/Dell/OneDrive/Desktop/charak_samita_app_sam/Datatset/Prescription.xlsx')


In [58]:
# Prepare the disease to treatment mapping
ayurvedic_treatment_dict = dict(
    zip(
        [str(disease).lower() for disease in ayurvedic_data['Diseases']],
        ayurvedic_data['ayurvedic_treatment']
    )
)


DATA PRE- PROCESSING

In [59]:
# Preprocess data
X = data['symptoms']
y = data['disease']


In [60]:
vectorizer = CountVectorizer(ngram_range=(1, 2))

MODEL PIPELINE AND MODEL FITTING

In [61]:
# Create a CountVectorizer and Naive Bayes pipeline
model = make_pipeline(vectorizer, MultinomialNB())


In [62]:
# Train the model
model.fit(X, y)

,steps,"[('countvectorizer', ...), ('multinomialnb', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


TAKING INPUT FROM USER FOR DISEASE PREDICTION

In [63]:
def preprocess_input(user_input):
    """ Preprocess user input for model prediction """
    return [" ".join(user_input.lower().split())]  # Join symptoms into a single string and convert to lowercase


In [64]:
def detect_disease(user_input):
    """ Detect disease based on user input symptoms """
    prediction = model.predict(preprocess_input(user_input))
    return prediction


MATCHING SYMPTOMS AND RECOMMENDING TREATMENT

In [65]:
def recommend_ayurvedic_treatment(disease):
    """ Recommend Ayurvedic treatment based on detected disease """
    disease = disease.lower()
    best_match_score = 0
    best_match_disease = None
    for ayurvedic_disease in ayurvedic_treatment_dict.keys():
        score = fuzz.partial_ratio(disease, ayurvedic_disease.lower())
        if score > best_match_score:
            best_match_score = score
            best_match_disease = ayurvedic_disease
    if best_match_disease is not None:
        return ayurvedic_treatment_dict[best_match_disease]
    else:
        return "No Ayurvedic treatment found for this disease."


DOCTOR'S LOCATION IN THEIR STATE

In [66]:
def find_nearby_ayurvedic_doctors(location):
    nearby_doctors = []
    
    # Read data from CSV file
    csv_filename = "doctors_data_location.csv"
    try:
        with open(csv_filename, mode='r', newline='') as file:
            reader = csv.reader(file)
            next(reader)  # Skip header row
            for row in reader:
                if location.lower() in row[1].lower():  # Check if location matches
                    nearby_doctors.append((row[0], float(row[2]), float(row[3])))  # Add doctor's data

    except FileNotFoundError:
        print(f"Error: The file {csv_filename} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return nearby_doctors


In [67]:
def on_submit():
    name = name_entry.get()
    age = age_entry.get()
    gender = gender_var.get()
    location = location_entry.get()
    user_input = symptoms_entry.get("1.0", tk.END)  # Get all text from the Text widget
    detected_disease = detect_disease(user_input)[0]
    recommendation = recommend_ayurvedic_treatment(detected_disease)
    
    
    result_text = f"Name: {name}\nAge: {age}\nGender: {gender}\nLocation: {location}\nSymptoms: {user_input}\nDetected Disease: {detected_disease}\nAyurvedic Recommendation: {recommendation}\n"
    
    messagebox.showinfo("Results", result_text)
    
    # Generate and save PDF report
    generate_pdf_report(name, age, gender, location, user_input, detected_disease, recommendation)
    
    
    # Get nearby doctors
    nearby_doctors = find_nearby_ayurvedic_doctors(location)
    # Display nearby doctors on map using Folium
    m = folium.Map(location=[nearby_doctors[0][1], nearby_doctors[0][2]], zoom_start=10)
    for doctor in nearby_doctors:
        folium.Marker([doctor[1], doctor[2]], popup=doctor[0]).add_to(m)
    m.save('nearby_doctors_map.html')
   
    
    # Open the map in a web browser
    os.system('start nearby_doctors_map.html')


PDF GENERATION

In [68]:
def generate_pdf_report(name, age, gender, location, user_input, detected_disease, recommendation):
    """ Generate and save a PDF report """
    try:
        folder_selected = filedialog.askdirectory()
        if folder_selected:
            filename = os.path.join(folder_selected, f"{name}_report.pdf")
            c = canvas.Canvas(filename, pagesize=letter)
            
            
            # Add logo
            logo_path = "C:/Users/Dell/OneDrive/Desktop/charak_samita_app_sam/Images/mlogo-removebg-preview.png"  # Adjust the path to your logo image
            c.drawImage(logo_path, 50, 700, width=150, height=70, preserveAspectRatio=True)
            
            # Add text content
            c.drawString(100, 640, "Patient Report")
            c.drawString(100, 620, f"Name: {name}")
            c.drawString(100, 600, f"Age: {age}")
            c.drawString(100, 580, f"Gender: {gender}")
            c.drawString(100, 560, f"Location: {location}")
            c.drawString(100, 540, f"Symptoms: {user_input}")
            c.drawString(100, 520, f"Detected Disease: {detected_disease}")
            c.drawString(100, 500, f"Ayurvedic Recommendation: {recommendation}")
            
            
            c.save()
            messagebox.showinfo("PDF Report Generated", f"PDF Report saved as {filename}")
        else:
            messagebox.showwarning("Warning", "No folder selected. PDF report not generated.")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {str(e)}")

FREQUENTLY ASKED QUESTIONS (FAQ) WINDOW

In [69]:
# ✅ Create the main application window
# Prevent Tkinter crash on rerun in Jupyter
try:
    root.destroy()
except:
    pass

root = tk.Tk()
root.title("Chakra Samhita")

# ✅ Define window dimensions BEFORE using them
window_width = 1530
window_height = 870

# ✅ Get screen width & height AFTER Tk() is created
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

# ✅ Set geometry properly before creating widgets
root.geometry(f"{window_width}x{window_height}+{int((screen_width - window_width) / 2)}+{int((screen_height - window_height) / 2)}")
root.configure(bg="#D1D3C7")


def open_faq_window():
    faq_window = tk.Toplevel(root)
    faq_window.title("FAQ")
    faq_window.geometry("1530x870")

    faq_label = tk.Label(faq_window, text="Frequently Asked Questions", font=("Helvetica", 20, "bold"), bg="white")
    faq_label.pack(pady=7)
    
    faq_text = tk.Text(faq_window, wrap="word", padx=10, pady=10, font=("Calibri", 13), bg="white", fg="black")
    faq_text.insert(tk.END, """
1. What is this application about?
This application provides recommendations for Ayurvedic treatments, predicts symptoms, and recommends doctors.

2. How do I use the application?
You can start by entering your name and other required information. Then, you can view the Ayurvedic recommendations, predicted symptoms, and recommended doctors.

3. Can I trust the recommendations provided?
The recommendations are based on general Ayurvedic principles. For personalized medical advice, please consult a healthcare professional.

4. Who can I contact for support?
For support, you can reach out to our customer service at support@example.com.

5. How do I exit the application?
You can exit the application by closing the window or clicking the 'Exit' button if available.

6. Is the information on this website accurate and reliable?
Yes, the information on this website is accurate and reliable. We work with experienced healthcare professionals to ensure that our information and recommendations are up-to-date and trustworthy.

7. How accurate is the disease predicting model?
The accuracy of the disease predicting model varies depending on the disease being predicted, but it is generally quite accurate. However, it is important to note that this model is not a substitute for a medical diagnosis.

8. What should I do if the model predicts that I have a disease?
If the model predicts that you may have a disease, we recommend that you seek the advice of a healthcare professional for a proper diagnosis and treatment plan.

9. Can I use this model to diagnose myself?
Unfortunately no., this model is not meant to be used as a substitute for a medical diagnosis. It is intended to provide information and suggestions, but you should always consult with a healthcare professional for a proper diagnosis and treatment plan.

10. Is Ayurvedic medicine safe?
When used properly, Ayurvedic medicine can be safe and effective. However, it is important to consult with a healthcare professional before using any new treatments or remedies, and to ensure that you are using them correctly.
""")
    faq_text.config(state=tk.DISABLED)
    faq_text.pack(expand=True, fill="both")


def faq():
    faq_button = tk.Button(root, text="FAQ", command=open_faq_window, bg="black", fg="white", width=15)
    faq_button.place(relx=0.95, rely=0.05, anchor="ne")


IMAGE FILES FOR UI

In [70]:
# Load the image
def display_image():
    try:
        img_path = r'C:/Users/Dell/OneDrive/Desktop/charak_samita_app_sam/Images/mlogo-removebg-preview.png'
        
        img = Image.open(img_path).convert("RGBA")  # ✅ Ensures transparency handling
        img = img.resize((100, 100), Image.Resampling.LANCZOS)  # ✅ Updated from ANTIALIAS
        
        global photo
        photo = ImageTk.PhotoImage(img)  # ✅ Prevents garbage collection
        
        image_label = tk.Label(root, image=photo, bg="#e6ebb3")
        image_label.place(x=10, y=10)
    
    except Exception as e:
        print("Image Error:", e)


In [71]:
def thumbnail():
    try:
        # Load and resize the banner image (reduced height for proper layout)
        img1 = Image.open(r'C:/Users/Dell/OneDrive/Desktop/charak_samita_app_sam/Images/thumbnail.jpg')
        img1 = img1.resize((1700, 230), Image.Resampling.LANCZOS)

        # Make copy and apply transparency
        img1_low_opacity = img1.copy()
        img1_low_opacity.putalpha(200)

        draw = ImageDraw.Draw(img1_low_opacity)

        # --- Title ---
        font = ImageFont.truetype("arialbd.ttf", 55)
        draw.text((500, 15), "Chakra Samhita", font=font, fill="black")

        # --- Sub Title 1 ---
        font1 = ImageFont.truetype("arial.ttf", 24)
        draw.text((400, 75), "LEADING-EDGE DISEASE PREDICTION TECHNOLOGY", font=font1, fill="black")

        # --- Sub Title 2 ---
        font2 = ImageFont.truetype("arialbd.ttf", 22)
        draw.text((450, 115), "YOUR GATEWAY TO AYURVEDIC HEALING", font=font2, fill="black")

        # Convert to PhotoImage and show
        global tphoto
        tphoto = ImageTk.PhotoImage(img1_low_opacity)
        tk.Label(root, image=tphoto, bg="#D1D3C7").place(x=0, y=0)

    except Exception as e:
        print("Error:", e)


In [72]:

def ayurvedic_recom():
    try:
        img2 = Image.open(r'C:/Users/Dell/OneDrive/Desktop/charak_samita_app_sam/Images/ayurvedic recom.png')
        img2 = img2.resize((350,200), Image.Resampling.LANCZOS)

        global aphoto
        aphoto = ImageTk.PhotoImage(img2)

        image2_label = tk.Label(root, image=aphoto, bg="white")
        image2_label.place(x=450, y=520)
    except Exception as e:
        print("Error:", e)


def predicting_symptoms():
    try:
        img3 = Image.open(r'C:/Users/Dell/OneDrive/Desktop/charak_samita_app_sam/Images/predicting symptoms.png')
        img3 = img3.resize((350,200), Image.Resampling.LANCZOS)

        global pphoto
        pphoto = ImageTk.PhotoImage(img3)

        image3_label = tk.Label(root, image=pphoto, bg="white")
        image3_label.place(x=30, y=520)
    except Exception as e:
        print("Error:", e)


def recom_doctors():
    try:
        img4 = Image.open(r'C:/Users/Dell/OneDrive/Desktop/charak_samita_app_sam/Images/recom doctors.png')
        img4 = img4.resize((350,200), Image.Resampling.LANCZOS)

        global rphoto
        rphoto = ImageTk.PhotoImage(img4)

        image4_label = tk.Label(root, image=rphoto, bg="white")
        image4_label.place(x=875, y=520)
    except Exception as e:
        print("Error:", e)


RUNNING APPLICATION

In [73]:
# Set the window size and background color
window_width = root.winfo_screenwidth()
window_height = root.winfo_screenheight()  # ✅ Dynamically match screen height

root.geometry(f"{window_width}x{window_height}+{int((screen_width - window_width) / 2)}+{int((screen_height - window_height) / 2)}")
root.configure(bg="#D1D3C7")

# Call functions to display images and UI components
thumbnail()
faq()
display_image()
ayurvedic_recom()
predicting_symptoms()
recom_doctors()

# Create a frame for user input, centered in the window
frame_width = 250
frame_height = 150
frame_border_width = 2
frame = tk.Frame(root, bg="white", width=frame_width, height=frame_height, bd=frame_border_width, relief=tk.SOLID)
frame.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

# Name
tk.Label(frame, text="Name:", bg="white").grid(row=0, column=0, padx=10, pady=5)
name_entry = tk.Entry(frame, bg="white", width=18)  # ✅ Smaller width
name_entry.grid(row=0, column=1, padx=5, pady=5)

# Age
tk.Label(frame, text="Age:", bg="white").grid(row=1, column=0, padx=10, pady=5)
age_entry = tk.Entry(frame, bg="white", width=18)  # ✅ Smaller width
age_entry.grid(row=1, column=1, padx=5, pady=5)

# Gender
tk.Label(frame, text="Gender:", bg="white").grid(row=2, column=0, padx=10, pady=5)
gender_var = tk.StringVar(value="Male")
gender_menu = tk.OptionMenu(frame, gender_var, "Male", "Female", "Other")
gender_menu.config(bg="white", width=15)  # ✅ Compact dropdown
gender_menu.grid(row=2, column=1, padx=5, pady=5)

# Location
tk.Label(frame, text="Location:", bg="white").grid(row=3, column=0, padx=10, pady=5)
location_entry = tk.Entry(frame, bg="white", width=18)  # ✅ Same size as others
location_entry.grid(row=3, column=1, padx=5, pady=5)

# Symptoms
tk.Label(frame, text="Symptoms:", bg="white").grid(row=4, column=0, padx=10, pady=5)
symptoms_entry = tk.Text(frame, width=20, height=3, bg="white")  # ✅ Small text box
symptoms_entry.grid(row=4, column=1, padx=5, pady=5)

# Submit button
submit_button = tk.Button(frame, text="Submit", width=12, command=on_submit)  # ✅ Reduced button size
submit_button.grid(row=5, column=0, columnspan=2, pady=5)

# Run the application
root.mainloop()


Error: The file doctors_data_location.csv was not found.


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Program Files\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Dell\AppData\Local\Temp\ipykernel_9604\2727166474.py", line 22, in on_submit
    m = folium.Map(location=[nearby_doctors[0][1], nearby_doctors[0][2]], zoom_start=10)
                             ~~~~~~~~~~~~~~^^^
IndexError: list index out of range
